### 1. Importing Libraries & Dataset

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
targets = pd.read_csv('target.csv')

### 2. Cleaning the data

#### a) Cleaning Numericals & Feature Selection

In [3]:
display(numerical.head())
display(numerical.shape)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

(95412, 315)

Below, there is not any null or duplicate values

In [4]:
nulls_percent_df = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage


In [5]:
sum(numerical.duplicated()) 

0

Below I wrote a code to check if there are any columns I can turn to categorical

In [6]:
def disc_cont(df):
    
    discrete_col = []
    continuous_col = []
    
    # if unique value count more than %0.00015 in a column -> continuous
    # else -> discrete
    
    for col in df.columns:
        if len(df[col].unique())/len(df[col]) > 0.00015:
            continuous_col.append(col)
        
        else:
            discrete_col.append(col)
            
    return discrete_col, continuous_col

In [7]:
disc_num, cont_num = disc_cont(numerical)
print('Discrete variables are: ', disc_num)

Discrete variables are:  ['INCOME', 'WEALTH1', 'WEALTH2', 'HV3', 'HV4', 'ANC6', 'MHUC2', 'HPHONE_D', 'RFA_2F']


In [8]:
to_categorical = numerical[['HV3', 'HV4', 'ANC6', 'MHUC2', 'HPHONE_D', 'RFA_2F']]

In [9]:
numerical = numerical.drop(columns=['HV3', 'HV4', 'ANC6', 'MHUC2', 'HPHONE_D', 'RFA_2F'], axis=1)

In [10]:
numerical.shape

(95412, 309)

#### Feature selection with Variance Treshold

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numerical)

In [12]:
pd.DataFrame(numerical_scaled).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308
0,0.000000,0.608247,0.666667,1.000000,0.000000,0.000000,0.393939,0.343434,0.181818,0.101010,0.020202,0.011494,0.555556,0.010051,0.011108,0.009378,0.0,0.353535,0.656566,0.474747,0.535354,0.929293,0.010101,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.113402,0.0,0.000000,0.000000,0.464286,0.571429,0.607143,0.476190,0.595238,0.642857,0.333333,0.313131,0.424242,0.272727,0.111111,0.141414,0.181818,0.171717,0.131313,0.111111,0.151515,0.121212,0.111111,0.343434,0.252525,0.181818,0.262626,0.101010,0.232323,0.181818,0.333333,0.494949,0.282828,0.121212,0.040404,0.616162,0.070707,0.164384,0.191919,0.304615,0.394286,0.979798,0.959596,0.020202,0.020202,0.000000,0.000000,0.070707,0.070707,0.000000,0.079833,0.105833,0.868687,0.141414,0.969697,0.040404,0.070707,0.383838,0.808081,0.707071,0.323232,0.848485,0.161616,0.060606,0.04,0.050505,0.090909,0.151515,0.030303,0.226667,0.505051,0.252525,0.000000,0.000000,0.000000,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.010101,0.616162,0.682353,0.677778,0.245902,0.100,0.020202,0.000000,0.000000,0.141414,0.010101,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.774120,0.204667,0.212000,0.232667,0.252000,0.073818,0.131313,0.232323,0.232323,0.232323,0.151515,0.010101,0.00,0.000000,0.010101,0.040404,0.252525,0.242424,0.262626,0.171717,0.020202,0.00,0.00000,0.020202,0.282828,0.040404,0.515152,0.010101,0.464646,0.545455,0.030303,0.888889,0.080808,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.040404,0.010101,0.131313,0.155556,0.210526,0.020202,0.454545,0.565657,0.646465,0.505051,0.646465,0.444444,0.626263,0.535354,1.000000,0.0,0.000000,0.090909,0.030303,0.080808,0.131313,0.090909,0.000000,0.054545,0.090909,0.030303,0.151515,0.191919,0.050505,0.040404,0.030303,0.0,0.030303,0.414141,0.010101,0.000000,0.070707,0.131313,0.060606,0.050505,0.000000,0.059701,0.090909,0.040404,0.010101,0.030303,0.101010,0.020202,0.010101,0.070707,0.787879,0.020202,0.000000,0.705882,0.161616,0.101010,0.393939,0.212121,0.216216,0.040404,0.030303,0.051546,0.202020,0.100000,0.263889,0.040404,0.000000,0.000000,0.000000,0.181818,0.393939,0.000000,0.343434,0.232323,0.181818,0.161616,0.012048,0.040404,0.000000,0.250000,0.000000,0.050505,0.018182,0.000000,0.000000,0.000000,0.000000,0.00,0.074074,0.0,0.030303,0.747475,0.888889,0.080808,0.000000,0.040404,0.969697,0.777778,0.191919,0.419355,0.596154,0.050505,0.141414,0.141414,0.313131,0.545455,0.464646,0.0,0.0,0.909091,0.000000,0.101010,0.000000,0.0,0.000000,0.333333,0.656566,0.404040,1.000000,1.000000,0.285714,0.101010,0.070707,0.433333,0.366492,0.315789,0.168831,0.023965,0.127119,0.341463,0.005,0.001401,0.010,0.003676,0.006465,0.498045,0.622951
1,0.000014,0.463918,0.833333,1.000000,0.066390,0.000000,0.151515,0.555556,0.111111,0.060606,0.020202,0.011494,1.000000,0.036585,0.039552,0.028190,1.0,0.000000,0.000000,0.505051,0.505051,

In [13]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))


# This drops the columns that have a variance less than this threshold
sel = sel.fit(numerical_scaled)
temp = sel.transform(numerical_scaled)
temp = pd.DataFrame(temp)
print(numerical_scaled.shape)
print(temp.shape)

(95412, 309)
(95412, 74)


In [14]:
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())

In [15]:
drop_num = [col[0] for col in zip(numerical.columns, var_list) if col[1] == False]  #list of low variance columns: to be dropped
drop_num

['TCODE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HU3',
 'HU4',
 'HHD1',
 'HHD4',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA4',
 'HUPA5',
 'HUPA7',
 'DMA',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC11',
 'IC1

In [16]:
 numerical = numerical.drop(columns=drop_num, axis=1)

In [17]:
numerical.columns

Index(['AGE', 'INCOME', 'WEALTH1', 'VIETVETS', 'WWIIVETS', 'WEALTH2',
       'POP90C1', 'POP90C2', 'POP90C3', 'ETH1', 'ETH2', 'HHN3', 'DW1', 'DW2',
       'DW4', 'DW5', 'DW6', 'HV1', 'HV2', 'HU1', 'HU2', 'HU5', 'HHD2', 'HHD3',
       'HHD5', 'ETHC2', 'HVP1', 'HVP2', 'HVP3', 'HVP4', 'HVP5', 'HVP6', 'HUR2',
       'HUPA2', 'HUPA3', 'HUPA6', 'RP1', 'RP2', 'RP3', 'RP4', 'MSA', 'ADI',
       'IC6', 'HHAS3', 'MC1', 'MC2', 'PEC2', 'TPE13', 'LFC2', 'LFC4', 'LFC6',
       'LFC7', 'LFC8', 'LFC9', 'VC1', 'VC3', 'POBC2', 'LSC1', 'VOC2', 'HC2',
       'HC4', 'HC5', 'HC6', 'HC7', 'HC8', 'HC11', 'HC13', 'HC17', 'HC18',
       'HC19', 'MHUC1', 'CARDPROM', 'CONTROLN', 'CLUSTER2'],
      dtype='object')

#### b) Cleaning Categoricals

In [18]:
categorical = pd.concat((categorical,to_categorical),axis=1)
print(categorical.shape)
categorical.head()

(95412, 28)


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,HV3,HV4,ANC6,MHUC2,HPHONE_D,RFA_2F
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,3,2,0,2,0,4
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,12,10,1,4,0,2
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1,2,1,0,2,1,4
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2,2,1,0,2,1,4
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3,4,3,0,2,1,2


In [19]:
for col in categorical.columns:
    categorical[col] = categorical[col].astype('object')
    print(categorical[col].value_counts())

other    30457
CA       17343
FL        8376
TX        7535
IL        6420
MI        5654
NC        4160
WA        3577
GA        3403
IN        2980
WI        2795
MO        2712
Name: STATE, dtype: int64
40    6295
35    3624
36    3617
27    3553
24    3539
49    3315
12    3000
18    2897
13    2706
30    2607
39    2601
45    2515
43    2385
11    2378
51    2322
14    2246
41    2176
44    1953
16    1947
2     1942
21    1891
8     1828
10    1804
46    1800
28    1630
17    1621
20    1568
53    1441
3     1428
34    1358
42    1358
23    1324
31    1285
22    1282
25    1275
38    1200
15    1173
1     1140
7     1076
5     1014
37     972
26     947
47     868
29     859
32     795
48     787
50     776
9      770
6      603
33     593
4      561
19     497
52     270
Name: CLUSTER, dtype: int64
H    74582
U    20830
Name: HOMEOWNR, dtype: int64
F        54234
M        39094
other     2084
Name: GENDER, dtype: int64
3    64829
2    23455
1     7128
Name: DATASRCE, dtype: int6

In [20]:
# I will drop RFA_2R because it i only 1 value
# I will drop  DOB_YR, DOB_MM because I have a column AGE
# I will drop ODATEW_MM,ODATEW_YR, MINRDATE_YR, MAXRDATE_YR, LASTDATE_YR, FIRSTDATE_YR
# date-wise I am more interested with the months to see the pattern, I will drop years

In [21]:
categorical = categorical.drop(columns=['RFA_2R', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR', 'DOB_MM', 'MINRDATE_YR', 'MAXRDATE_YR',
       'LASTDATE_YR','FIRSTDATE_YR'], axis=1)

In [22]:
categorical.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM,FIRSTDATE_MM,HV3,HV4,ANC6,MHUC2,HPHONE_D,RFA_2F
0,IL,36,H,F,3,E,C,T,2,8,2,12,11,3,2,0,2,0,4
1,CA,14,H,M,3,G,A,S,1,10,12,12,10,12,10,1,4,0,2
2,NC,43,U,M,3,E,C,R,2,11,7,12,1,2,1,0,2,1,4
3,CA,44,U,F,3,E,C,R,2,11,11,12,2,2,1,0,2,1,4
4,FL,16,H,F,3,F,A,S,2,10,1,1,3,4,3,0,2,1,2


In [23]:
all_data = pd.concat((categorical,numerical,targets),axis=1)
print(all_data.shape)
all_data.head()

(95412, 95)


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM,FIRSTDATE_MM,HV3,HV4,ANC6,MHUC2,HPHONE_D,RFA_2F,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,ETH2,HHN3,DW1,DW2,DW4,DW5,DW6,HV1,HV2,HU1,HU2,HU5,HHD2,HHD3,HHD5,ETHC2,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR2,HUPA2,HUPA3,HUPA6,RP1,RP2,RP3,RP4,MSA,ADI,IC6,HHAS3,MC1,MC2,PEC2,TPE13,LFC2,LFC4,LFC6,LFC7,LFC8,LFC9,VC1,VC3,POBC2,LSC1,VOC2,HC2,HC4,HC5,HC6,HC7,HC8,HC11,HC13,HC17,HC18,HC19,MHUC1,CARDPROM,CONTROLN,CLUSTER2,TARGET_B,TARGET_D
0,IL,36,H,F,3,E,C,T,2,8,2,12,11,3,2,0,2,0,4,60.000000,5,9,34,18,5,0,35,65,92,1,49,97,95,2,0,0,479,635,86,14,7,80,70,84,50,2,7,13,27,47,0,61,0,0,0,2,5,17,73,0.0,177.0,13,51,46,54,13,45,64,64,62,53,99,0,34,18,74,88,77,31,14,14,31,54,46,90,10,33,65,40,6,27,95515,39,0,0.0
1,CA,14,H,M,3,G,A,S,1,10,12,12,10,12,10,1,4,0,2,46.000000,6,9,55,11,9,99,0,0,67,0,75,99,99,0,0,0,5468,5218,96,4,9,94,88,95,44,97,98,98,98,99,94,83,0,0,0,91,91,91,94,4480.0,13.0,2,66,56,44,2,71,83,81,64,57,99,99,55,11,39,50,92,10,26,56,97,99,0,96,4,99,0,99,20,12,148535,1,0,0.0
2,NC,43,U,M,3,E,C,R,2,11,7,12,1,2,1,0,2,1,4,61.611649,3,1,29,33,1,0,2,98,96,2,44,69,69,6,5,3,497,546,78,22,18,76,65,86,60,0,1,6,18,50,0,36,2,24,3,0,2,9,44,0.0,281.0,32,31,38,62,21,61,73,70,78,62,82,99,28,32,84,96,65,22,12,23,50,69,31,0,35,22,77,17,9,26,15078,60,0,0.0
3,CA,44,U,F,3,E,C,R,2,11,11,12,2,2,1,0,2,1,4,70.000000,1,4,14,31,0,0,8,92,61,0,50,85,83,4,1,0,1000,1263,48,52,6,73,61,84,36,10,25,50,69,92,10,42,0,9,0,1,8,17,34,9340.0,67.0,27,26,46,54,17,43,81,68,33,31,0,99,14,31,67,56,43,38,10,19,39,45,55,45,17,23,77,22,16,27,172556,41,0,0.0
4,FL,16,H,F,3,F,A,S,2,10,1,1,3,4,3,0,2,1,2,78.000000,3,2,9,53,9,99,0,0,2,98,57,99,98,0,0,0,576,594,90,10,0,82,49,92,1,0,1,2,16,67,0,45,0,0,0,25,58,74,83,5000.0,127.0,24,11,20,80,1,76,73,65,80,31,81,99,9,53,65,99,45,34,1,3,6,33,67,9,72,99,1,21,6,43,7112,26,0,0.0


I will save my cleaned data to use later at my linear regression model

In [54]:
all_data.to_csv('all_data.csv', index = False)
categorical.to_csv('categorical_c.csv',index = False)
numerical.to_csv('numerical_c.csv',index = False)

In [25]:
# going to drop TARGET_D column for today
data = all_data.drop(columns=['TARGET_D'], axis=1)

#### Oversampling

- I know there is imbalance in the data, I will oversample it
- I will oversample only my train set

In [26]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [27]:
# X/y split
X = data.drop('TARGET_B',axis = 1)
y = data['TARGET_B']

In [28]:
# train-test-split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # default fraction is .2

In [29]:
X_train_imb = pd.DataFrame(X_train)

In [30]:
trainset = pd.concat((X_train, y_train), axis=1)

In [31]:
trainset['TARGET_B'].value_counts()

0    67970
1     3589
Name: TARGET_B, dtype: int64

- oversampling the train data

In [32]:
from sklearn.utils import resample

category_0 = trainset[trainset['TARGET_B'] == 0]
category_1 = trainset[trainset['TARGET_B'] == 1]

In [33]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [34]:
train_upsampled = pd.concat([category_1_oversampled, category_0], axis=0)

- above my train upsampled data is in an order, I will mix them below
- because I will do cross validatyion in future

In [35]:
train_upsampled = train_upsampled.sample(frac =1)

In [36]:
train_upsampled['TARGET_B'].value_counts()

1    67970
0    67970
Name: TARGET_B, dtype: int64

- now again I will do X-y split to my trainset because I will one hot encode my categoricals and scale numericals
- I don't need to scale numericals for Random Forest but I will try logistic regression model too, so preferred to do it now

In [37]:
X_train = train_upsampled.drop('TARGET_B',axis = 1)
y_train = train_upsampled['TARGET_B']

In [38]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [39]:
#splitting into categorical and numerical
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)

X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(include = object)

- for both scaling and encoding, I will fit it to my train set, use the same transformer to my test set

In [40]:
#scale numericals
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) 
numericals_train_scaled = transformer.transform(X_train_num)
numericals_test_scaled = transformer.transform(X_test_num)

In [41]:
#encode categoricals
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_cat)
categoricals_train_encoded = encoder.transform(X_train_cat).toarray()
categoricals_test_encoded = encoder.transform(X_test_cat).toarray()

In [42]:
X_train = np.concatenate((numericals_train_scaled,categoricals_train_encoded),axis=1)
X_test = np.concatenate((numericals_test_scaled,categoricals_test_encoded),axis=1)
X_train = pd.DataFrame(X_train)
X_train.shape

(135940, 247)

- I will make a logistic regression model

In [43]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0, solver='saga')
LR.fit(X_train, y_train)
LR.score(X_test, y_test) 

0.6073869114996018

In [44]:
pred = LR.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[13821,  8778],
       [  587,   667]], dtype=int64)

In [45]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print("precision: ",precision_score(y_test,pred))    # true pos / true pos+false pos
print("recall: ",recall_score(y_test,pred))    #recall is most important, true pos/true pos + false neg
print("f1: ",f1_score(y_test,pred))

precision:  0.07061937533086289
recall:  0.5318979266347688
f1:  0.12468454995793998


- I will make a RandomForestClassifier model

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))


0.6234000294247463
0.5744350815411059


0    22599
1     1254
Name: TARGET_B, dtype: int64

array([[12987,  9612],
       [  539,   715]], dtype=int64)

In [47]:
print("precision: ",precision_score(y_test,y_pred))    # true pos / true pos+false pos
print("recall: ",recall_score(y_test,y_pred))    #recall is most important, true pos/true pos + false neg
print("f1: ",f1_score(y_test,y_pred))

precision:  0.06923598334463058
recall:  0.5701754385964912
f1:  0.12347811069855799


In [48]:
# I will save the confusion matrix for later
random_forest_conf_mat = confusion_matrix(y_test, y_pred)

- I cross validate my Random Forest Classifier
- it is 0.61 on every group

In [49]:
from sklearn.model_selection import cross_validate

results = cross_validate(clf,X_train, y_train, cv = 5)

In [50]:
print(results['test_score'])
print(results['test_score'].mean())

[0.61652199 0.62255407 0.62472414 0.61755186 0.61409445]
0.6190893041047522


In [51]:
# I will save this score for later
random_forest_accuracy = results['test_score'].mean()

- I will create a pipeline to compare my models

In [52]:
#complete the code here
model1 = clf
model2 = LR

import numpy as np


model_pipeline = [model1, model2]
model_names = ['RandomForestClassifier', 'Logistic Regression']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_name] = mean_score
print(scores)

{'RandomForestClassifier': 0.6190304546123289, 'Logistic Regression': 0.6030822421656613}


- When I want to choose between my classification models I considered 2 things
- 1- accuracy 2- precision score
- I am choosing RandomForestClassifier model and pickle it to use later

In [53]:
import pickle
pickle.dump(clf,open('clf.p', 'wb'))
pickle.dump(random_forest_conf_mat,open('confusion_matrix.p', 'wb'))
pickle.dump(random_forest_accuracy,open('ran_for_accuracy.p', 'wb'))
